# QCD Lattice Field Theory: Monte Carlo Methods

This notebook demonstrates the implementation of Monte Carlo methods for lattice field theory, following the approach outlined in "Quantum Chromodynamics on the Lattice" and the Creutz article.

## Overview

We implement and compare:
1. **Metropolis algorithm** for Gaussian distribution (as a warmup)
2. **1D scalar field theory** using Metropolis algorithm
3. **Hybrid Monte Carlo (HMC)** for 1D field theory
4. **Comparison** of methods and analysis of autocorrelation times

## Theoretical Background

### 1D Scalar Field Theory

We consider a 1D scalar field $\phi(x)$ on a lattice with action:

$$S[\phi] = \sum_{x} \left[ \frac{1}{2}(\phi(x+1) - \phi(x))^2 + \frac{1}{2}m^2\phi(x)^2 + \lambda\phi(x)^4 \right]$$

where:
- First term: kinetic energy (discrete derivative)
- Second term: mass term  
- Third term: quartic self-interaction

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sys
import os

# Set up path to src directory
sys.path.append('../src')

# Import our implementations
from metropolis import MetropolisGaussian
from field_theory_1d import FieldTheory1D
from hmc import HMCFieldTheory1D
from utils import *

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## 1. Metropolis Algorithm for Gaussian Distribution

We start with a simple example: sampling from a Gaussian distribution $N(\mu, \sigma^2)$ using the Metropolis algorithm.

### Algorithm:
1. Start with initial configuration $x_0$
2. Propose new state: $x' = x + \delta x$ where $\delta x \sim U(-\epsilon, \epsilon)$
3. Accept with probability: $\min(1, \exp(-(S[x'] - S[x])))$
4. Repeat

For Gaussian: $S[x] = \frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2$ (up to constants)

In [ ]:
# Create Metropolis sampler for Gaussian distribution
sampler = MetropolisGaussian(mu=2.0, sigma=1.5)

# Generate samples
print("Generating samples from Gaussian distribution...")
samples = sampler.sample(n_samples=10000, step_size=1.0, burn_in=1000)

# Print diagnostics
sampler.print_diagnostics()

# Plot results
sampler.plot_results()

### Effect of Step Size

The step size $\epsilon$ affects the acceptance rate and mixing:
- **Too small**: High acceptance, but slow mixing
- **Too large**: Low acceptance, inefficient sampling
- **Optimal**: ~50% acceptance rate

In [ ]:
# Demonstrate step size effect
from metropolis import demonstrate_step_size_effect
demonstrate_step_size_effect()

## 2. 1D Field Theory with Metropolis Algorithm

Now we implement the Metropolis algorithm for a 1D scalar field theory on a lattice.

### Algorithm:
1. Start with random field configuration $\phi(x)$
2. **Sweep**: For each lattice site, propose new field value
3. Accept/reject based on local action change
4. Measure observables after each sweep

### Key observables:
- $\langle\phi\rangle$: Average field value
- $\langle\phi^2\rangle$: Field variance
- $\langle\phi^4\rangle$: Fourth moment
- Two-point correlation function: $\langle\phi(0)\phi(r)\rangle$

In [ ]:
# Create 1D field theory instance
field_theory = FieldTheory1D(
    lattice_size=50,
    mass_squared=0.5,
    lambda_coupling=0.1
)

print("Running 1D field theory simulation with Metropolis algorithm...")

# Run simulation
results = field_theory.run_simulation(
    n_sweeps=10000,
    step_size=0.5,
    burn_in=2000,
    measurement_interval=1
)

# Analyze results
field_theory.analyze_results()

In [ ]:
# Plot field configuration
field_theory.plot_field_configuration()

# Plot observables evolution
field_theory.plot_observables()

# Plot correlation function
field_theory.plot_correlation_function()

### Parameter Dependence

Let's explore how the field behavior depends on the parameters $m^2$ and $\lambda$:

In [ ]:
# Compare different parameter settings
from field_theory_1d import compare_parameters
compare_parameters()

## 3. Hybrid Monte Carlo (HMC) Implementation

HMC combines molecular dynamics with Monte Carlo to achieve better performance, especially for critical slowing down.

### Algorithm:
1. **Refresh momenta**: $p_i \sim N(0,1)$ for each site
2. **Molecular dynamics**: Evolve $(\phi, p)$ using Hamilton's equations:
   - $\dot{\phi}_i = \frac{\partial H}{\partial p_i} = p_i$
   - $\dot{p}_i = -\frac{\partial H}{\partial \phi_i} = -\frac{\partial S}{\partial \phi_i}$
3. **Metropolis accept/reject**: Based on change in Hamiltonian

### Hamiltonian:
$$H = \frac{1}{2}\sum_i p_i^2 + S[\phi]$$

### Leapfrog Integration:
$$p_{n+1/2} = p_n + \frac{\epsilon}{2}F_n$$
$$\phi_{n+1} = \phi_n + \epsilon p_{n+1/2}$$
$$p_{n+1} = p_{n+1/2} + \frac{\epsilon}{2}F_{n+1}$$

where $F_i = -\frac{\partial S}{\partial \phi_i}$

In [ ]:
# Create HMC field theory instance
hmc = HMCFieldTheory1D(
    lattice_size=50,
    mass_squared=0.5,
    lambda_coupling=0.1
)

print("Running 1D field theory simulation with HMC algorithm...")

# Run HMC simulation
hmc_results = hmc.run_hmc_simulation(
    n_trajectories=2000,
    step_size=0.1,
    n_md_steps=10,
    burn_in=400
)

# Analyze results
hmc.analyze_hmc_results()

In [ ]:
# Plot HMC diagnostics
hmc.plot_hmc_diagnostics()

## 4. Comparison: Metropolis vs HMC

Let's compare the performance of Metropolis and HMC algorithms by analyzing their autocorrelation times.

In [ ]:
# Direct comparison
from hmc import compare_hmc_metropolis
compare_hmc_metropolis()

### Detailed Autocorrelation Analysis

Let's analyze the autocorrelation properties in more detail:

In [ ]:
# Analyze autocorrelation for both methods
metropolis_phi2 = np.array(results['observables']['phi_squared'])
hmc_phi2 = np.array(hmc_results['observables']['phi_squared'])

# Compute autocorrelation times
tau_metropolis = integrated_autocorrelation_time(metropolis_phi2)
tau_hmc = integrated_autocorrelation_time(hmc_phi2)

print(f"Autocorrelation time comparison for ⟨φ²⟩:")
print(f"Metropolis: {tau_metropolis:.2f}")
print(f"HMC:        {tau_hmc:.2f}")
print(f"Improvement factor: {tau_metropolis/tau_hmc:.2f}")

# Effective sample sizes
eff_metropolis = effective_sample_size(metropolis_phi2)
eff_hmc = effective_sample_size(hmc_phi2)

print(f"\nEffective sample sizes:")
print(f"Metropolis: {eff_metropolis:.1f}")
print(f"HMC:        {eff_hmc:.1f}")

# Plot autocorrelation functions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Metropolis autocorrelation
lags_metro, autocorr_metro = autocorrelation_function(metropolis_phi2, max_lag=100)
ax1.plot(lags_metro, autocorr_metro, 'b-', label='Metropolis')
ax1.axhline(y=1/np.e, color='r', linestyle='--', alpha=0.5, label='1/e')
ax1.set_xlabel('Lag')
ax1.set_ylabel('Autocorrelation')
ax1.set_title('Metropolis Autocorrelation')
ax1.legend()
ax1.grid(True, alpha=0.3)

# HMC autocorrelation
lags_hmc, autocorr_hmc = autocorrelation_function(hmc_phi2, max_lag=100)
ax2.plot(lags_hmc, autocorr_hmc, 'g-', label='HMC')
ax2.axhline(y=1/np.e, color='r', linestyle='--', alpha=0.5, label='1/e')
ax2.set_xlabel('Lag')
ax2.set_ylabel('Autocorrelation')
ax2.set_title('HMC Autocorrelation')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. HMC Parameter Optimization

HMC performance depends on:
- **Step size** $\epsilon$: Too small → slow, too large → high rejection
- **Number of MD steps** $N$: Determines trajectory length $L = \epsilon N$
- **Trajectory length** $L$: Should be $\sim 1$ for optimal performance

In [ ]:
# Optimize HMC parameters
from hmc import optimize_hmc_parameters
optimize_hmc_parameters()

## 6. Error Analysis

Let's perform a proper error analysis using jackknife and bootstrap methods:

In [ ]:
# Error analysis for key observables
print("Error Analysis for ⟨φ²⟩ observable:")
print("=" * 35)

# Metropolis results
metropolis_mean, metropolis_jack_err = jackknife_error(metropolis_phi2)
metropolis_boot_mean, metropolis_boot_err = bootstrap_error(metropolis_phi2)

print(f"\nMetropolis:")
print(f"  Mean: {metropolis_mean:.6f}")
print(f"  Jackknife error: {metropolis_jack_err:.6f}")
print(f"  Bootstrap error: {metropolis_boot_err:.6f}")

# HMC results
hmc_mean, hmc_jack_err = jackknife_error(hmc_phi2)
hmc_boot_mean, hmc_boot_err = bootstrap_error(hmc_phi2)

print(f"\nHMC:")
print(f"  Mean: {hmc_mean:.6f}")
print(f"  Jackknife error: {hmc_jack_err:.6f}")
print(f"  Bootstrap error: {hmc_boot_err:.6f}")

# Compare statistical errors
print(f"\nStatistical error comparison:")
print(f"  Metropolis / HMC (jackknife): {metropolis_jack_err / hmc_jack_err:.2f}")
print(f"  Metropolis / HMC (bootstrap): {metropolis_boot_err / hmc_boot_err:.2f}")

## 7. Binning Analysis

Binning analysis helps estimate the true statistical error by accounting for autocorrelations:

In [ ]:
# Binning analysis
print("Binning Analysis:")
print("=" * 20)

# Metropolis binning
plot_binning_analysis(metropolis_phi2, title="Metropolis Binning Analysis")

# HMC binning
plot_binning_analysis(hmc_phi2, title="HMC Binning Analysis")

## 8. Summary and Conclusions

### Key Findings:

1. **Metropolis Algorithm**: Simple to implement, good for understanding MCMC basics
2. **HMC Algorithm**: More complex but significantly better performance for field theory
3. **Autocorrelation**: HMC typically shows much lower autocorrelation times
4. **Parameter Optimization**: Critical for HMC performance

### Performance Comparison:
- **Autocorrelation time**: HMC typically 2-10x better than Metropolis
- **Effective sample size**: HMC generates more independent samples
- **Statistical errors**: HMC achieves better precision for same computational cost

### Next Steps:
1. Implement critical slowing down studies
2. Extend to 2D field theory
3. Add gauge fields for full QCD
4. Implement advanced algorithms (RHMC, multi-level, etc.)

In [ ]:
print("FINAL PERFORMANCE SUMMARY")
print("="*50)

print(f"\n1D Field Theory (N={field_theory.N}, m²={field_theory.m_squared}, λ={field_theory.lambda_coupling})")
print(f"\nMetropolis Algorithm:")
print(f"  Acceptance rate: {results['acceptance_rate']:.3f}")
print(f"  Autocorrelation time: {tau_metropolis:.2f}")
print(f"  Effective sample size: {eff_metropolis:.1f}")
print(f"  Statistical error: {metropolis_jack_err:.6f}")

print(f"\nHMC Algorithm:")
print(f"  Acceptance rate: {hmc_results['acceptance_rate']:.3f}")
print(f"  Autocorrelation time: {tau_hmc:.2f}")
print(f"  Effective sample size: {eff_hmc:.1f}")
print(f"  Statistical error: {hmc_jack_err:.6f}")

print(f"\nImprovement Factors (HMC vs Metropolis):")
print(f"  Autocorrelation time: {tau_metropolis/tau_hmc:.2f}x better")
print(f"  Effective sample size: {eff_hmc/eff_metropolis:.2f}x better")
print(f"  Statistical precision: {metropolis_jack_err/hmc_jack_err:.2f}x better")

